In [ ]:
import os
import numpy
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
    GENERE = 'pp'

    TIMESTAMP_LOCATION = '2timestamps/'+GENERE+'/'
    OUTPUT_LOCATION = '3spectrs_raw/'+GENERE+'/'+GENERE



    OUTPUT_CHUNKS_SIZE = 100
    N_INPUT_FILES = 7400        #3700



    FRAME_SIZE = 2048
    HOP_SIZE = 512
    sr = 22050

In [ ]:
#---------------------------------------------------------------------------------------GET S_SCALE
#   ESTRAZIONE SHORT TIME FOURIER TRANSFORM (S_scale???) per ogni audio
#il risultato e' un array bidimensionale (frequenza-numero di frames con quella frequenza)

def getSScales(audios):
    S_scale = []
    for x in audios:
        S_scale.append(librosa.stft(x, n_fft=FRAME_SIZE, hop_length=HOP_SIZE))
    return S_scale

#---------------------------------------------------------------------------------------GET Y_SCALE
#ESTRAZIONE SPETTROGRAMMA
# output: lista di float 
def getYScale(SScale):
    Y_scale = []
    for x in SScale:
        Y_scale.append(np.abs(x) ** 2)
        #print(Y_scale.shape)
        #print(type(Y_scale[0][0]))
    return Y_scale
#---------------------------------------------------------------------------------------GET Y LOG SCALE
#input -> lista, output -> numpy array
def getYLogScale(YScale):
    Y_log_scale = []
    for x in YScale:
        Y_log_scale.append(librosa.power_to_db(x))
        # np.concatenate((Y_log_scale,librosa.power_to_db(x)))
        # Y_log_scale.append(librosa.power_to_db(x))
    return Y_log_scale
#---------------------------------------------------------------------------------------GET Y LOG SCALE FROM AUDIO 
#riceve in input una lista di audio, e produce in output la relativa ylog scale
def getYLogScaleFromAudio(audios):
    S_scale = getSScales(audios)
    Yscale = getYScale(S_scale)
    YLogScale = getYLogScale(Yscale)
    YLogScale = numpy.array(YLogScale[0:len(audios)])   #se va tenere questo
    # YLogScale = numpy.array(YLogScale[0:len(audios)-1])
    # YLogScale = numpy.array(YLogScale[0:NFILES-1])

    return YLogScale
#-------------------------------------------------------------------------------------VISUALIZZAZIONE DELLO SPETTROGRAMMA
# def plot_spectrogram(Y, sr, hop_length, y_axis="linear",title=""):
#     plt.figure(figsize=(25, 10))
#     plt.title(title)
#     librosa.display.specshow(Y, 
#                              sr=sr, 
#                              hop_length=hop_length, 
#                              x_axis="time", 
#                              y_axis=y_axis)
#     plt.colorbar(format="%+2.f")


In [ ]:
for n_chunk in range (N_INPUT_FILES//OUTPUT_CHUNKS_SIZE):
    ts_chunk = []   #timestamps chunk

    for n_file in range(1,OUTPUT_CHUNKS_SIZE+1):

        #carico il timestamp
        # print('carico :'+TIMESTAMP_LOCATION+str(n_file+100*n_chunk))
        timestamp = np.load(TIMESTAMP_LOCATION+str(n_file+100*n_chunk)+'.npy')

        #lo aggiungo al chunk
        ts_chunk.append(timestamp)
    
    #al termine converto il chunk di timestamps a spettrogrammi e lo salvo
    spectrs_chunk = getYLogScaleFromAudio(ts_chunk)
    np.save(OUTPUT_LOCATION+str(n_chunk),spectrs_chunk)
    print(f'salvo il chunk {n_chunk}')

In [ ]:
print('DONE')